In [1]:
import numpy as np
import pandas as pd

In [99]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.svm import SVR

In [37]:
car=pd.read_csv("E:/cdac/Project-End-Cdac-4Members/after_iqr.csv")
car_data=car.copy()

In [38]:
car_data.head()

,Price,abtest,vehicleType,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,postalCode,Car_age
0,480.0,test,sedan,Manual,40,golf,150000,1,Petrol,volkswagen,No,70435,23
1,18300.0,test,coupe,Manual,190,golf,125000,5,Diesel,audi,Yes,66954,5
2,9800.0,test,suv,Automatic,163,grand,125000,8,Diesel,jeep,No,90480,12
3,1500.0,test,small_car,Manual,75,golf,150000,6,Petrol,volkswagen,No,91074,15
4,3600.0,test,small_car,Manual,69,fabia,90000,7,Diesel,skoda,No,60437,8


In [39]:
car_data.shape

(360748, 13)

In [40]:
def remove_outliers(df: pd.DataFrame) -> pd.DataFrame:
    # Calculate the 10th and 90th percentiles (Q1 and Q3)
    Q1 = df.quantile(0.1, numeric_only=True)
    Q3 = df.quantile(0.9, numeric_only=True)
    IQR = Q3 - Q1

    # Filter the DataFrame to only include columns that are in the IQR calculation
    df_filtered = df[Q1.index]

    # Apply the outlier removal condition
    filtered_df = df_filtered[~((df_filtered < (Q1 - 1.5 * IQR)) | (df_filtered > (Q3 + 1.5 * IQR))).any(axis=1)]

    # Return the original DataFrame without outliers, including non-numeric columns
    return df.loc[filtered_df.index]

# Assuming `new_car` is your DataFrame
clean_user_car = remove_outliers(car_data)

In [89]:
#----------------------------Index Added-----------------------

In [83]:
clean_car['Index'] = range(1, len(clean_car) + 1)

In [87]:
csv_filename = 'final_data354205.csv'
clean_car.to_csv(csv_filename, index=False)

In [90]:
#--------------------------------------

In [41]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ["notRepairedDamage", "vehicleType",
            "abtest", "gearbox", "model", "fuelType", "brand"] # Initialize a list of categorical column names

encoder = LabelEncoder() # Initialize a LabelEncoder object
encoded_user_car = clean_user_car.copy() # Make a copy of the clean_user_car dataframe and assign it to encoded_user_car
encoded_user_car[cat_cols] = encoded_user_car[cat_cols].apply(encoder.fit_transform) # Apply the fit_transform method of the encoder object to the categorical columns of the encoded_user_car dataframe


In [42]:
encoded_user_car.head()

,Price,abtest,vehicleType,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,postalCode,Car_age
0,480.0,1,4,1,40,117,150000,1,6,38,0,70435,23
1,18300.0,1,3,1,190,117,125000,5,1,1,1,66954,5
2,9800.0,1,7,0,163,118,125000,8,1,14,0,90480,12
3,1500.0,1,5,1,75,117,150000,6,6,38,0,91074,15
4,3600.0,1,5,1,69,102,90000,7,1,31,0,60437,8


In [61]:
X = encoded_user_car.drop(["Price"], axis=1)
y = encoded_user_car["Price"]

In [62]:
encoded_user_car.shape

(354205, 13)

In [63]:
X.head()

,abtest,vehicleType,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,postalCode,Car_age
0,1,4,1,40,117,150000,1,6,38,0,70435,23
1,1,3,1,190,117,125000,5,1,1,1,66954,5
2,1,7,0,163,118,125000,8,1,14,0,90480,12
3,1,5,1,75,117,150000,6,6,38,0,91074,15
4,1,5,1,69,102,90000,7,1,31,0,60437,8


In [64]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)
print(f"Train Shape : {X_train.shape} --------- Test Shape : {X_test.shape}")

Train Shape : (283364, 12) --------- Test Shape : (70841, 12)


In [47]:
from sklearn.metrics import r2_score

scores = {"r2_score" : make_scorer(r2_score), "mae" : make_scorer(mean_absolute_error), "mse" : make_scorer(mean_squared_error)}

#XGBoost-Regressor

In [48]:
model = xgb.XGBRegressor(tree_method="hist", n_estimators = 1000)
result = cross_validate(model, X_train, y_train, cv = 5, verbose = 2, scoring=scores)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   9.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.5s remaining:    0.0s


[CV] END .................................................... total time=  10.3s
[CV] END .................................................... total time=  10.8s
[CV] END .................................................... total time=  11.1s
[CV] END .................................................... total time=  11.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   53.7s finished


In [49]:
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f"R2 Score : {r2_score(y_test, pred)}")

R2 Score : 0.8948905941588821


In [50]:
import pickle

# Assuming 'model' is your trained model
filename = 'XGBoost-model.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(model, file)

from IPython.display import FileLink

# Provide a link to download the file
FileLink('XGBoost-model.pkl')



C:\Users\Admin\XGBoost-model.pkl

--------------------------------------XGBoost using Standard Scalar------------------------------------------------

In [53]:
from sklearn.preprocessing import StandardScaler
# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data and transform it
scaled_data = scaler.fit_transform(X)

# Convert the scaled data back into a DataFrame for easy viewing
scaled_df = pd.DataFrame(scaled_data, columns=X.columns)
print("\nDataFrame after Standard Scaling:")
scaled_df.head()


DataFrame after Standard Scaling:


,abtest,vehicleType,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,postalCode,Car_age
0,0.964959,-0.162972,0.506217,-1.277229,0.223932,0.600099,-1.378200,0.665865,1.295840,-0.325697,0.757835,1.598730
1,0.964959,-0.805910,0.506217,1.277161,0.223932,-0.042903,-0.251802,-1.526575,-1.473913,3.070341,0.622635,-1.184557
2,0.964959,1.765841,-1.975438,0.817371,0.238297,-0.042903,0.592996,-1.526575,-0.500756,-0.325697,1.536368,-0.102168
3,0.964959,0.479966,0.506217,-0.681205,0.223932,0.600099,0.029797,0.665865,1.295840,-0.325697,1.559438,0.361714
4,0.964959,0.479966,0.506217,-0.783381,0.008459,-0.943105,0.311397,-1.526575,0.771833,-0.325697,0.369520,-0.720676


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)
print(f"Train Shape : {X_train.shape} --------- Test Shape : {X_test.shape}")

Train Shape : (283364, 12) --------- Test Shape : (70841, 12)


In [55]:
scores = {"r2_score" : make_scorer(r2_score), "mae" : make_scorer(mean_absolute_error), "mse" : make_scorer(mean_squared_error)}

In [56]:
model = xgb.XGBRegressor(tree_method="hist", n_estimators = 1000)
result = cross_validate(model, X_train, y_train, cv = 5, verbose = 2, scoring=scores)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=  10.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s remaining:    0.0s


[CV] END .................................................... total time=  10.1s
[CV] END .................................................... total time=  10.4s
[CV] END .................................................... total time=  10.9s
[CV] END .................................................... total time=  11.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   53.2s finished


In [57]:
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(f"R2 Score : {r2_score(y_test, pred)}")

R2 Score : 0.8948905941588821


----------------------------------------------------------------

--------------------------Linear Regression------------------------

In [65]:
modellr =LinearRegression() 

modellr.fit(X_train, y_train)


LinearRegression()

In [66]:
# Predict on the test data
y_pred = modellr.predict(X_test)

# Calculate R² (accuracy for regression)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Linear Regression:")
print(f"R² (Accuracy): {r2:.4f}")

Linear Regression:
R² (Accuracy): 0.6055


In [77]:
new=np.array([1,4,1,40,117,15000,1,6,38,0,70435,23])
# Reshape to 2D array (1 observation, 12 features)
new_data= new.reshape(1,-1)

In [78]:
modellr.predict(new_data)

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([5430.56290978])

In [91]:
# Assuming 'model' is your trained model
filename = 'Linear-Regression-model.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(modellr, file)

from IPython.display import FileLink

# Provide a link to download the file
FileLink('Linear-Regression-model.pkl')


C:\Users\Admin\Linear-Regression-model.pkl

In [72]:
X.shape

(354205, 12)

#Decision-Tree-Regressor---

In [93]:
# Initialize the model
model = DecisionTreeRegressor(random_state=1)

# Train the model
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [94]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate R² Score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

R² Score: 0.792013979951945


In [96]:
# Assuming 'model' is your trained model
filename = 'Decisiontree-model.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(model, file)

from IPython.display import FileLink

# Provide a link to download the file
FileLink('Decisiontree-model.pkl')


C:\Users\Admin\Decisiontree-model.pkl

#-----------------Time bhetla tar------------------------------------

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the grid search
grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=5)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best estimator to make predictions
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Evaluate the tuned model
r2_best = r2_score(y_test, y_pred_best)
print(f"Tuned R² Score: {r2_best}")


---------------------------------------------------------------------

In [97]:
#------------------SVR----------------------------

In [100]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data
X_test_scaled = scaler.transform(X_test)

# Initialize the SVR model
modelsvr = SVR(kernel='rbf') 

# Train the model
modelsvr.fit(X_train_scaled, y_train)


SVR()

In [ ]:
# Make predictions on the test set
y_pred = modelsvr.predict(X_test_scaled)

# Calculate R² Score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")


In [ ]:
# Assuming 'model' is your trained model
filename = 'SVR-model.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(model, file)

from IPython.display import FileLink

# Provide a link to download the file
FileLink('SVR-model.pkl')


In [ ]:
#--------------------------Random-Forest-Regressor-------------------------

In [ ]:
# Initialize the Random Forest Regressor
modelrfr = RandomForestRegressor(n_estimators=100, random_state=1)

# Train the model
modelrfr.fit(X_train, y_train)


In [ ]:
# Make predictions on the test set
y_pred = modelrfr.predict(X_test)
# Calculate R² Score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

In [ ]:
# Assuming 'model' is your trained model
filename = 'RFR-model.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(modelrfr, file)

from IPython.display import FileLink

# Provide a link to download the file
FileLink('RFR-model.pkl')


In [ ]:
-----------------Time Bhetl tar----------------------------------

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the grid search
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best estimator to make predictions
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Evaluate the tuned model
r2_best = r2_score(y_test, y_pred_best)
print(f"Tuned R² Score: {r2_best}")


In [ ]:
#--------------------------------------------------------------------------

In [ ]:
#----------------------------KNN-Regressor----------------------------

In [ ]:
# Initialize the KNN Regressor
modelknn = KNeighborsRegressor(n_neighbors=5)

# Train the model
modelknn.fit(X_train, y_train)

# Make predictions on the test set
y_pred = modelknn.predict(X_test)

# Calculate R² Score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")


In [ ]:
# Assuming 'model' is your trained model
filename = 'KNN-model.pkl'

# Save the model to disk
with open(filename, 'wb') as file:
    pickle.dump(modelknn, file)

from IPython.display import FileLink

# Provide a link to download the file
FileLink('KNN-model.pkl')


In [ ]:
#-------------Time bhetl tar Hyper-parameter tuning---------------------------

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],  # 'uniform' means all neighbors are equally weighted, 'distance' weights by distance
    'metric': ['euclidean', 'manhattan']
}

# Initialize the grid search
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Use the best estimator to make predictions
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Evaluate the tuned model
r2_best = r2_score(y_test, y_pred_best)
print(f"Tuned R² Score: {r2_best}")


In [ ]:
#---------------------------------------------------------------

In [ ]:
#---------------Neural-Networks---------------------------------

In [ ]:
# Define the model
model = Sequential()

# Input layer
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))  # Dropout layer with 30% dropout rate

# Hidden layers
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))  # Dropout layer with 30% dropout rate

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))  # Dropout layer with 30% dropout rate

# Output layer
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with EarlyStopping
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1, callbacks=[early_stopping])

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}")

# Predicting and calculating R² score
y_pred = model.predict(X_test)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")


In [ ]:
Paths=["E:/cdac/Project-End-Cdac-4Members/Models/XGBoost-model.pkl","rfr-model.pkl",""]

In [ ]:
# Load the model
loaded_model = joblib.load('E:/cdac/Project-End-Cdac-4Members/Models/XGBoost-model.pkl","rfr-model.pkl')

In [ ]:
#Give here the values you need
new=np.array([1,4,1,40,117,15000,1,6,38,0,70435,23])
# Reshape to 2D array (1 observation, 12 features)
new_data= new.reshape(1,-1)
# Make predictions and evaluate
loaded_model.predict(new)

In [ ]:
#calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")